### Analyses
- all neuronal pairwise correlations during sleep deprivation go here

### Imports


In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import subjects
from neuropy import plotting

### Distribution of pairwise correlations in 2.5h windows 

In [ ]:
sessions = subjects.pf_sess()

pcorr_df = []
for sub, sess in enumerate(tqdm(sessions)):
    epochs = sess.get_zt_epochs(include_maze=True)
    neurons = sess.neurons_stable.get_neuron_type("pyr")
    wave_similarity = neurons.get_waveform_similarity()
    pairs_bool = wave_similarity < 1  # only pairs which are least similar

    get_corr = lambda epoch: (
        neurons.time_slice(epoch[0], epoch[1])
        .get_binned_spiketrains(bin_size=0.25, ignore_epochs=sess.artifact)
        .get_pairwise_corr(pairs_bool=pairs_bool)
    )

    pre = epochs["MAZE"].flatten()
    pre_corr = get_corr([pre[0], pre[1]])

    ev, rev = [], []
    for e in epochs[1:].itertuples():
        e_corr = get_corr([e.start, e.stop])
        pcorr_df.append(pd.DataFrame(dict(pair_corr=e_corr, zt=e.label, grp=sess.tag)))

pcorr_df = pd.concat(pcorr_df, ignore_index=True)
# subjects.GroupData().save(ev_df, "ev_in_chunks")

In [ ]:
from plotters import violinplot
from statannotations.Annotator import Annotator

_, ax = plt.subplots()

# pcorr_df = pcorr_df[np.abs(pcorr_df.pair_corr)<200]
plot_kw = dict(
    data=pcorr_df,
    x="zt",
    y="pair_corr",
    hue="grp",
    hue_order=["NSD", "SD"],
    ax=ax,
    bw=1,
)
violinplot(**plot_kw)

orders = pcorr_df.zt.unique()

# Across groups
# pairs = [((_, "NSD"), (_, "SD")) for _ in orders] + [
# (("0-2.5", "NSD"), ("5-7.5", "SD"))
# ]
pairs = [(("0-2.5", "NSD"), ("2.5-5", "NSD"))] + [(("0-2.5", "SD"), ("2.5-5", "SD"))]


annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
annotator.configure(test="Wilcoxon", color="#4AB33E")
annotator.apply_and_annotate()
annotator.reset_configuration()

### Distribution of IN-PN correlations in 2.5h windows 

In [ ]:
sessions = subjects.pf_sess()

pcorr_df = []
for sub, sess in enumerate(tqdm(sessions)):
    epochs = sess.get_zt_epochs(include_maze=True)
    neurons = sess.neurons_stable.get_neuron_type(["pyr", "inter"])
    n_type_bool = (neurons.neuron_type == "inter").astype("int")

    pairs_bool = n_type_bool[:, np.newaxis] - n_type_bool[np.newaxis, :]

    get_corr = lambda epoch: (
        neurons.time_slice(epoch[0], epoch[1])
        .get_binned_spiketrains(bin_size=0.25, ignore_epochs=sess.artifact)
        .get_pairwise_corr(pairs_bool=pairs_bool)
    )

    ev, rev = [], []
    for e in epochs.itertuples():
        e_corr = get_corr([e.start, e.stop])
        pcorr_df.append(pd.DataFrame(dict(pair_corr=e_corr, zt=e.label, grp=sess.tag)))

pcorr_df = pd.concat(pcorr_df, ignore_index=True)
# subjects.GroupData().save(ev_df, "ev_in_chunks")

In [ ]:
from plotters import violinplot
from statannotations.Annotator import Annotator

_, ax = plt.subplots()

plot_kw = dict(
    data=pcorr_df, x="zt", y="pair_corr", hue="grp", hue_order=["NSD", "SD"], ax=ax
)
violinplot(**plot_kw)

orders = pcorr_df.zt.unique()

# Across groups
pairs = [((_, "NSD"), (_, "SD")) for _ in orders] + [
    (("0-2.5", "NSD"), ("5-7.5", "SD"))
]
annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
annotator.configure(test="t-test_welch", color="#4AB33E")
annotator.apply_and_annotate()
annotator.reset_configuration()

### Compare pariwise correlation using scatter plot from first to last hour of sleep deprivation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from neuropy.plotting import Fig
from scipy.ndimage import gaussian_filter1d
import subjects

sd_sessions = subjects.sd.allsess[2:-1]
nsd_sessions = subjects.nsd.ratNday2 + subjects.nsd.ratSday2 + subjects.nsd.ratUday2
print(sd_sessions)

In [ ]:
pcorr_sd = []
pcorr_nsd = []
for grp_ind, sessions in enumerate([sd_sessions, nsd_sessions]):
    for sub, sess in enumerate(sessions):
        maze = sess.paradigm["maze"]
        rec_duration = sess.eegfile.duration
        neurons = sess.neurons.get_neuron_type("mua")

        bin_size = 60
        pcorr_sub = []
        for start in [maze[0], maze[0] + 4 * 3600]:
            pcorr = (
                neurons.time_slice(start, start + 3600)
                .get_binned_spiketrains(bin_size=bin_size)
                .get_pairwise_corr()
            )
            pcorr_sub.append(pcorr)
        pcorr_sub = np.asarray(pcorr_sub).T
    if grp_ind == 0:
        pcorr_sd.append(pcorr_sub)

    if grp_ind == 1:
        pcorr_nsd.append(pcorr_sub)

pcorr_sd = np.vstack(pcorr_sd)
pcorr_nsd = np.vstack(pcorr_nsd)

In [ ]:
%matplotlib inline
figure = Fig()
fig,gs = figure.draw(grid=(2,2),style='Pres')

for i,pcorr in enumerate([pcorr_sd,pcorr_nsd]):
    ax = plt.subplot(gs[i])
    ax.scatter(pcorr[:,0],pcorr[:,1],s=20,c='gray',marker='o',edgecolors='w')
    # ax.hexbin(pcorr[:,0],pcorr[:,1],cmap='inferno',gridsize=50)
    ax.axline((0,0),slope=1)
    ax.set_xlabel('First hour')
    ax.set_ylabel('Fifth hour')

### Pairwise correlation across time each session individually

In [ ]:
from neuropy.analyses import neurons_correlation

sessions = subjects.pf_sess()
pair_corr_all = []
time_all = []
names = []
for sub, sess in enumerate(tqdm(sessions)):
    zt_epochs = sess.get_zt_epochs()
    t1, t2 = zt_epochs["PRE"].flatten()[0], zt_epochs["5-7.5"].flatten()[1]
    neurons = sess.neurons_stable.get_neuron_type("pyr").time_slice(t1, t2)
    name = sess.filePrefix.name.split("_")[:2]
    pair_corr = neurons_correlation.corr_across_time_window(neurons)
    time = (
        np.linspace(t1, t2, pair_corr.shape[0]) - zt_epochs["0-2.5"].flatten()[0]
    ) / 3600
    np.fill_diagonal(pair_corr, val=0)
    time_all.append(time)
    pair_corr_all.append(pair_corr)
    names.append("".join(name))

In [ ]:
from scipy.ndimage import gaussian_filter

fig = plotting.Fig(grid=(8, 4))
# _,axs = plt.subplots(7,2)
# axs = axs.reshape(-1)

# rat_names = ['Rat N','Rat S','Rat U']
for i, pair_corr in enumerate(pair_corr_all):
    ax = fig.subplot(fig.gs[i])
    # t = np.arange(pair_corr.shape[0])*(5/60)
    t = time_all[i]
    ax.pcolormesh(t, t, pair_corr, cmap="jet", rasterized=True, shading="auto")
    # ax.set_xticks([0,5,10])
    # ax.set_yticks([0,5,10])
    ax.set_xticks([-1, 0, 2.5, 5, 7.5])
    ax.set_yticks([-1, 0, 2.5, 5, 7.5])
    ax.set_title(f"{names[i]}")

# fig.suptitle('Window = 300 s, Bin size = 0.25 s')
fig.savefig(subjects.figpath_sd / "correlation_across_time")

### Pairwise correlation across pooled across sessions

In [ ]:
sessions = subjects.pf_sess()
# sessions = subjects.Sd().ratJday1
corr_sd, corr_nsd = [], []
for sub, sess in enumerate(sessions):
    pre = sess.epochs.pre
    maze1 = sess.epochs.maze
    if maze1 is None:
        maze1 = sess.epochs.maze1
    post = sess.epochs.post
    tag = sess.recinfo.animal.tag

    # --- break region into periods --------
    # bins = sess.utils.getinterval([post[0], post[0] + 5 * 3600], 2)
    # sess.spikes.stability.firingRate(periods=bins)
    spks = sess.spikes.pyr
    epochs = np.arange(post[0], post[0] + 8 * 3600, 300)
    corr = []
    for i in range(len(epochs) - 1):
        corr.append(
            sess.spikes.corr.pairwise(spikes=spks, period=[epochs[i], epochs[i + 1]])
        )
    corr = np.asarray(corr)

    if tag == "sd":
        corr_sd.append(corr)
    else:
        corr_nsd.append(corr)


def process_(corr_mat):
    corr_mat = np.hstack(corr_mat)
    corr_mask = np.ma.array(corr_mat, mask=np.isnan(corr_mat))
    corr_ = np.ma.corrcoef(corr_mask)
    np.fill_diagonal(corr_, 0)
    # corr_ = gaussian_filter(corr_, sigma=1)
    return corr_


corr_sd = process_(corr_sd)
corr_nsd = process_(corr_nsd)

time = np.linspace(0, 8, corr_sd.shape[0] + 1)

figure = Fig()
fig, gs = figure.draw(num=1, grid=(4, 3), wspace=0.3)
axsd = plt.subplot(gs[2])
im = axsd.pcolormesh(time[:-1], time[:-1], corr_sd, cmap="Spectral_r", rasterized=True)
axsd.set_xlabel("Time (h)")
axsd.set_ylabel("Time (h)")
cbar = fig.colorbar(im, ax=axsd)
cbar.outline.set_linewidth(1)


axnsd = plt.subplot(gs[5])
im = axnsd.pcolormesh(
    time[:-1], time[:-1], corr_nsd, cmap="Spectral_r", rasterized=True
)
axnsd.set_xlabel("Time (h)")
cbar = fig.colorbar(im, ax=axnsd)
cbar.outline.set_linewidth(1)
# cbar.set_label("correlation")

figure.savefig("correlation_time_window", __file__)

### Mean pairwise correlation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from neuropy.utils import signal_process
import subjects
import pandas as pd
from neuropy.analyses import neurons_correlation

# sessions = (
#     subjects.sd.ratNday1
#     + subjects.sd.ratSday3
#     + subjects.sd.ratRday2
#     + subjects.sd.ratUday4
# )

sessions = subjects.nsd.ratNday2 + subjects.nsd.ratSday2 + subjects.nsd.ratUday2

In [ ]:
from neuropy.utils import mathutil

pcorr_df = pd.DataFrame()
for sub, sess in enumerate(sessions):
    neurons = sess.neurons.get_neuron_type("pyr")
    post = sess.paradigm["post"]
    sd = [post[0], post[0] + 5 * 3600]

    t_starts = np.arange(0, 5 * 3600 / 600) * 600 + sd[0]

    pcorr_sub = np.zeros(len(t_starts))
    for i, start in enumerate(t_starts):
        pair_corr = (
            neurons.time_slice(start, start + 600)
            .get_binned_spiketrains(bin_size=0.25)
            .get_pairwise_corr()
        )
        pcorr_sub[i] = np.nanmean(pair_corr)

    pcorr_df = pcorr_df.append(
        pd.DataFrame(
            {
                "bins": (t_starts - sd[0]) / 3600,
                "corr": mathutil.min_max_scaler(pcorr_sub),
                "sub": sub,
            }
        ),
        ignore_index=True,
    )

In [ ]:
%matplotlib widget

sns.lineplot(data=pcorr_df,x='bins',y='corr')

### Pairwise correlations between Zt0-5 vs Zt5-end

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import subjects

sessions = subjects.nsd.pf_sess + subjects.sd.pf_sess

In [ ]:
pair_corr_df = []
for sub, sess in enumerate(sessions):
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    e1 = [post[0], post[0] + 5 * 3600]
    e2 = [post[0] + 5 * 3600, post[1]]

    neurons = sess.neurons.get_neuron_type("pyr")
    e1_frate = neurons.time_slice(*e1).firing_rate
    e2_frate = neurons.time_slice(*e2).firing_rate

    good_indices = np.logical_and(e1_frate > 0, e2_frate > 0)
    neurons = neurons[good_indices]

    pair_corr = []
    for e in [e1, e2]:
        pair_corr.append(
            neurons.time_slice(*e).get_binned_spiketrains(0.25).get_pairwise_corr()
        )

    df = pd.DataFrame(dict(e1=pair_corr[0], e2=pair_corr[1], grp=sess.tag))
    pair_corr_df.append(df)

pair_corr_df = pd.concat(pair_corr_df, ignore_index=True)

subjects.GroupData().save(pair_corr_df, "post_first5_last5_paircorr")

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 

_,axs = plt.subplots(1,2,sharex=True,sharey=True)
axs = axs.reshape(-1)
for i, grp in enumerate(['NSD','SD']):
    df = pair_corr_df[pair_corr_df['grp']==grp]
    linreg = stats.linregress(df['e1'],df['e2'])
    sns.scatterplot(data=df,x='e1',y='e2',hue='grp',ax=axs[i])
    axs[i].set_title(f'r={linreg.rvalue.round(2)}, pvalue={linreg.pvalue}')


### Autocorrelation of correlation matrix during Zt0-5 of SD vs NSD
 - get a correlation of pairwise correlations only during Zt0-5 and perform a autocorrelation on this matrix. It was done to find if there is a shorter decay curve of SD compared to NSD.
 - Tried both 2d correlation and autocorrelation of each row, the result looks like a blob, nothing interesting here.

In [ ]:
import numpy as np
import subjects
import pandas as pd

sessions = subjects.nsd.ratSday2 + subjects.sd.ratSday3

In [ ]:
import pingouin as pg

corr = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type("pyr")
    windows = np.arange(post[0], post[0] + 5 * 3600, 300)
    pair_corr = []
    for w in windows[:-1]:
        pair_corr.append(
            neurons.time_slice(w, w + 300).get_binned_spiketrains().get_pairwise_corr()
        )
    pair_corr = np.array(pair_corr).T
    df = pd.DataFrame(pair_corr, columns=np.arange(len(windows) - 1))
    corr.append(df.corr().values)
subjects.GroupData().save(corr, "remaze_corr_across_session")

In [ ]:
%matplotlib widget
import scipy.signal as sg
import matplotlib.pyplot as plt

_,axs = plt.subplots(2,2)
# axs = axs.reshape(-1)

for i,c in enumerate(corr):
    np.fill_diagonal(c,0)
    ac = []
    for c_ in c: 
        ac.append(sg.correlate(c_,c_,mode='same'))
    ac = np.array(ac)
    axs[0,i].imshow(c,cmap='jet')
    axs[1,i].imshow(ac,cmap='jet')



### Pairwise correlation (Zt0-5) quantiles temporal evolution durign recovery sleep 

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
import subjects

sessions = subjects.sd.pf_sess

In [ ]:
template_corr, pair_corr = [], []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type("pyr")
    template = [post[0] + 2.5 * 3600, post[0] + 5 * 3600]

    template_corr.append(
        neurons.time_slice(*template).get_binned_spiketrains().get_pairwise_corr()
    )

    windows = np.arange(post[0] + 5 * 3600, post[0] + 8 * 3600, 300)

    pair_corr_sub = []
    for w in windows[:-1]:
        pair_corr_sub.append(
            neurons.time_slice(w, w + 300).get_binned_spiketrains().get_pairwise_corr()
        )
    pair_corr.append(np.array(pair_corr_sub).T)

template_corr = np.concatenate(template_corr)
pair_corr = np.vstack(pair_corr)
# subjects.GroupData().save(corr, "t_corr_across_session")

In [ ]:
quantiles = pd.qcut(template_corr, 5, labels=False)

mean_corr = []
mean_template = []
for i in range(5):
    mean_corr.append(np.nanmean(pair_corr[quantiles == i, :], axis=0))
    mean_template.append(np.nanmean(template_corr[quantiles == i]))

mean_corr = np.array(mean_corr)
mean_template = np.array(mean_template)
ratio = (mean_corr - mean_template[:, np.newaxis]) / mean_template[:, np.newaxis]

In [ ]:
%matplotlib widget

for i,mc in enumerate(ratio):
    plt.plot(mc,label=i)

plt.legend()

### Distribution of pairwise correlations during Zt0-5 comaprison SD vs NSD

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
import subjects

sessions = subjects.nsd.pf_sess + subjects.sd.pf_sess

In [ ]:
pair_corr_df = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type("pyr")
    e1 = [post[0], post[0] + 5 * 3600]
    e2 = [post[0] + 5 * 3600, post[1]]

    pcorr_e1 = neurons.time_slice(*e1).get_binned_spiketrains().get_pairwise_corr()
    pcorr_e2 = neurons.time_slice(*e2).get_binned_spiketrains().get_pairwise_corr()

    pair_corr_df.append(
        pd.DataFrame(dict(pcorr_first5=pcorr_e1, pcorr_last5=pcorr_e2, grp=sess.tag))
    )

pair_corr_df = pd.concat(pair_corr_df, ignore_index=True)
# subjects.GroupData().save(corr, "t_corr_across_session")

In [ ]:
%matplotlib widget
import seaborn as sns

sns.histplot(
    data=pair_corr_df,
    x="pcorr_first5",
    hue="grp",
    stat="probability",
    fill=False,
    element="step",
    common_bins=True,
    common_norm=False,
    palette=subjects.colors_sd(1),
    # kde=True,
    # cumulative=True,
)

sns.histplot(
    data=pair_corr_df,
    x="pcorr_last5",
    hue="grp",
    stat="probability",
    fill=False,
    element="step",
    common_bins=True,
    common_norm=False,
    palette=subjects.colors_sd(1.7),
    # log_scale=True,
    # cumulative=True,
)

plt.legend(['NSD_Zt0-5','SD_Zt0-5','NSD_Zt5-end','SD_Zt5-end'])
plt.xlim([-0.05,0.17])


### Pairwise correlations NSD early sleep vs SD recovery sleep
- Are they different ?
- Comapring distribution of pairwise correlations
- Results: Observed that during recovery sleep negative correlations are pushed towards zero. This may be an effect of higher delta oscillations during recovery sleep i.e, neurons are more synchronized during recovery sleep. 

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
import subjects

sessions = subjects.nsd.pf_sess + subjects.sd.pf_sess

In [ ]:
pair_corr_df = []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type("pyr")

    if sess.tag == "NSD":
        period = [post[0], post[0] + 4 * 3600]
        label = f"{sess.tag} Zt0-4 h POST"
    else:
        period = [post[0] + 5 * 3600, post[1]]
        label = f"{sess.tag} rec. slp."

    pcorr_125 = (
        neurons.time_slice(*period)
        .get_binned_spiketrains(bin_size=0.125)
        .get_pairwise_corr()
    )
    pcorr_250 = (
        neurons.time_slice(*period)
        .get_binned_spiketrains(bin_size=0.250)
        .get_pairwise_corr()
    )

    pair_corr_df.append(
        pd.DataFrame(dict(pcorr_125=pcorr_125, pcorr_250=pcorr_250, grp=sess.tag))
    )
pair_corr_df = pd.concat(pair_corr_df, ignore_index=True)
# subjects.GroupData().save(corr, "t_corr_across_session")

In [ ]:
%matplotlib widget
import seaborn as sns
from neuropy.plotting import Fig

fig = Fig(grid=(2, 1), fontsize=10)
ax = fig.subplot(fig.gs[0])

bins = np.linspace(-0.1, 0.3, 200)
sns.histplot(
    data=pair_corr_df,
    x="pcorr_250",
    hue="grp",
    stat="probability",
    fill=False,
    element="step",
    # common_bins=True,
    bins=bins,
    common_norm=False,
    hue_order=["NSD", "SD"],
    palette=subjects.colors_sd(1),
    # kde=True,
    # cumulative=True,
    lw=2,
    ax=ax,
)


ax.set_xlabel("Pairwise correlations")
ax.set_title(
    "Distribution of pairwise correlations between regular sleep and recovery sleep\n"
)

sns.histplot(
    data=pair_corr_df,
    x="pcorr_125",
    hue="grp",
    stat="probability",
    fill=False,
    element="step",
    # common_bins=True,
    bins=bins,
    common_norm=False,
    hue_order=["NSD", "SD"],
    palette=subjects.colors_sd(1.7),
    # kde=True,
    # cumulative=True,
    lw=2,
    ax=ax,
)


ax.legend(frameon=False)
fig.legend(
    ax,
    ["NSD Zt0-4 h", "SD recovery sleep"],
    subjects.colors_sd(1),
    fontsize=10,
    dy=0.05,
)
# ax.axvline(np.nanmean(pair_corr_df[pair_corr_df["grp"] == "NSD"].pcorr.mean()),color='k')
# ax.axvline(np.nanmean(pair_corr_df[pair_corr_df["grp"] == "SD"].pcorr.mean()),color='r')
# ax.set_ylabel("Pairwise correlations")
# plt.xscale('log')
# plt.yscale('log')
# plt.legend(['NSD_Zt0-5','SD_Zt0-5','NSD_Zt5-end','SD_Zt5-end'])
# plt.xlim([-0.05,0.17])
# stats.kruskal(
#     pair_corr_df[pair_corr_df["grp"] == "NSD"].pcorr,
#     pair_corr_df[pair_corr_df["grp"] == "NSD"].pcorr,
#     nan_policy="omit",
# )


### CCG of pairs with negative correlations
- Check what does CCG of a negative pairwise correlation look like.
- It seems, negative pairs have stronger dip in 20 to 100 ms region comapred to baseline (200 ms).
- One observation is that negative correlation pairs are have higher theta modulation.


In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
import subjects

sessions = subjects.sd.pf_sess

In [ ]:
from neuropy.utils.neurons_util import calculate_neurons_ccg

pair_corr, ccgs_all = [], []
for sub, sess in enumerate(sessions):
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type("pyr")
    frate_bool = neurons.firing_rate > 0
    neurons = neurons[frate_bool]
    e1 = [post[0] + 5 * 3600, post[1]]

    pcorr_e1 = (
        neurons.time_slice(*e1)
        .get_binned_spiketrains(bin_size=0.125)
        .get_pairwise_corr()
    )
    ccgs, t = calculate_neurons_ccg(neurons, bin_size=0.001, window_size=1)
    indices = np.tril_indices(neurons.n_neurons, k=-1)
    ccgs = ccgs[indices[0], indices[1], :]

    pair_corr.append(pcorr_e1)
    ccgs_all.append(ccgs)

pair_corr = np.concatenate(pair_corr)
ccgs_all = np.vstack(ccgs_all)

sort_ind = np.argsort(pair_corr)
pair_corr = pair_corr[sort_ind]
ccgs_all = ccgs_all[sort_ind, :]
ccgs_norm = ccgs_all / np.max(ccgs_all, axis=1, keepdims=True)

In [ ]:
%matplotlib widget
import seaborn as sns

nt = ccgs_all.shape[1]
center = nt//2
_,[[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2)

ax1.scatter(pair_corr,ccgs_all[:,center],s=3,c='gray')
ax1.set_yscale('log')

ccg_zscore =stats.zscore(ccgs_norm,axis=1)  
good_indices = ~np.isnan(np.sum(ccg_zscore,axis=1))
ax2.imshow(ccg_zscore[good_indices,:],aspect='auto',cmap='jet',vmin=0,vmax=0.5,extent=[t[0],t[-1],0,len(np.where(good_indices)[0])])
ax2.set_ylabel('Pairs')
ax2.set_xlabel('Time (ms)')

neg_ind = np.where(pair_corr<0)[0]
pos_ind = np.where(pair_corr>0)[0]

pos_mean =np.nanmean(ccgs_norm[pos_ind,:],axis=0) 
neg_mean =np.nanmean(ccgs_norm[neg_ind,:],axis=0) 
ax3.plot(t,stats.zscore(neg_mean),'#FF4081')
ax3.plot(t,stats.zscore(pos_mean),'#00C853')
ax3.set_ylabel('Normalized count (zscored)')
ax3.set_xlabel('Time (ms)')
ax3.legend(['negative','positive'])
ax3.set_title('Mean CCG of negative and positively correlated pairs')


### Reactivation of pairwise correlation quantiles during POST
- Does each quantile of correlation have different decay rate ?
- Maybe best understood if all correlations from all the sessions are pooled together.

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import subjects

sessions = subjects.sd.pf_sess

In [ ]:
pair_corr_all = []
for sub, sess in enumerate(sessions):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    neurons = sess.neurons.get_neuron_type("pyr")
    post_bins = np.arange(post[0], post[0] + 5 * 3600, 300)
    post_epochs = np.vstack((post_bins[:-1], post_bins[1:])).T

    all_epochs = np.vstack([pre, maze, post_epochs])

    pair_corr = []
    for e in all_epochs:
        pair_corr.append(
            neurons.time_slice(*e).get_binned_spiketrains(0.25).get_pairwise_corr()
        )
    pair_corr = np.array(pair_corr).T

    pair_corr_all.append(pair_corr)

pair_corr_all = np.vstack(pair_corr_all)

In [ ]:
%matplotlib widget
import pingouin as pg

_,axs = plt.subplots(1,2)
axs = axs.reshape(-1)

pre_corr = pair_corr_all[:,0]
maze_corr = pair_corr_all[:,1]
quantiles = pd.qcut(maze_corr,5,labels=False)
# quantiles = pd.qcut(pre_corr,5,labels=False)

ev_quantile = []
quantile_boundary = []
for q in range(5):
    quantile_boundary.append(pair_corr_all[quantiles==q,1].max())
    mat = pair_corr_all[quantiles==q,:]
    df = pd.DataFrame(mat,columns=np.arange(mat.shape[1]))
    
    ev = []
    for y in range(2,61):
        stats = pg.partial_corr(data=df,x=1,y=y,covar=0)
        ev.append(stats.r.values[0]**2)
    
    ev_quantile.append(np.array(ev))
    # mat_corr = df.corr().values
    # np.fill_diagonal(mat_corr,0)

    # axs[q].imshow(mat_corr,cmap='jet')
axs[0].plot(np.array(ev_quantile).T)

bins = np.arange(-0.1,0.3,0.01)
hist_paircorr = np.histogram(maze_corr[~np.isnan(maze_corr)],bins=bins)[0]
hist_pre = np.histogram(pre_corr[~np.isnan(pre_corr)],bins=bins)[0]
axs[1].plot(bins[:-1],hist_paircorr/hist_paircorr.sum())
axs[1].plot(bins[:-1],hist_pre/hist_pre.sum())
# for val in quantile_boundary[:-1]:
#     axs[1].axvline(val)

### Manifold/PCA trajectory during NSD and SD
- Although interesting patterns are visible during NSD and SD but not sure what to make of it.

In [ ]:
sessions = subjects.nsd.ratSday2 + subjects.sd.ratSday3

In [ ]:
import umap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import Isomap

reducer = umap.UMAP(n_components=2, n_neighbors=30)
dim_red = []
for sub, sess in enumerate(sessions):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")

    period = [post[0], post[0] + 5 * 3600]
    # period = maze
    # period = [post[0]+5*3600,post[1]]
    binspk = neurons.time_slice(*period).get_binned_spiketrains(0.5).spike_counts
    # pca = PCA(n_components=2)
    scaled_data = StandardScaler().fit_transform(binspk.T)
    dim_red.append(reducer.fit_transform(scaled_data))

In [ ]:
import matplotlib as mpl

cmap = mpl.cm.get_cmap("gist_rainbow")
_, axs = plt.subplots(1, 2)

for i, transform in enumerate(dim_red):
    ax = axs[i]
    n_t = transform.shape[0]
    colors = [cmap(_ / n_t) for _ in range(n_t)]
    x, y = transform.T
    ax.scatter(x, y, s=2)
    # ax.plot(x,y)